# Five Laws Cognitive Governance Validator Tutorial

This tutorial demonstrates how to use the Five Laws Validator tool to evaluate AI-generated content against SIM-ONE's Five Laws of Cognitive Governance.

## Overview

The Five Laws Validator evaluates text against:
1. **Architectural Intelligence** - Intelligence from coordination, not brute force
2. **Cognitive Governance** - Governed processes over unconstrained generation
3. **Truth Foundation** - Absolute truth principles over probabilistic drift
4. **Energy Stewardship** - Computational efficiency and resource awareness
5. **Deterministic Reliability** - Consistent, predictable outcomes

## Setup

First, let's import the necessary modules and set up our environment.

In [ ]:
import subprocess
import json
import sys
from pathlib import Path

# Path to the tool
TOOL_PATH = Path("../code/tools/run_five_laws_validator.py")

def validate_text(text, strictness="moderate", format="json"):
    """Helper function to validate text using the Five Laws Validator"""
    result = subprocess.run(
        ["python", str(TOOL_PATH), "--text", text, "--strictness", strictness, "--format", format],
        capture_output=True,
        text=True
    )
    return json.loads(result.stdout) if format == "json" else result.stdout

## Example 1: Basic Validation

Let's validate a simple AI response.

In [ ]:
# Example AI response to validate
response = """Climate change is supported by overwhelming scientific evidence from multiple 
independent sources including atmospheric measurements, ice core data, and global temperature 
records. The consensus among climate scientists exceeds 97%."""

# Validate the response
result = validate_text(response)

# Display results
print(f"Overall Compliance: {result['scores']['overall_compliance']:.1f}%")
print(f"Status: {result['pass_fail_status']}")
print(f"\nIndividual Law Scores:")
print(f"  Law 1 (Architectural Intelligence): {result['scores']['law1_architectural_intelligence']:.1f}%")
print(f"  Law 2 (Cognitive Governance): {result['scores']['law2_cognitive_governance']:.1f}%")
print(f"  Law 3 (Truth Foundation): {result['scores']['law3_truth_foundation']:.1f}%")
print(f"  Law 4 (Energy Stewardship): {result['scores']['law4_energy_stewardship']:.1f}%")
print(f"  Law 5 (Deterministic Reliability): {result['scores']['law5_deterministic_reliability']:.1f}%")

## Example 2: Validation with Different Strictness Levels

The validator supports three strictness levels: lenient, moderate, and strict.

In [ ]:
response = "AI systems can be helpful tools for various tasks."

for strictness in ["lenient", "moderate", "strict"]:
    result = validate_text(response, strictness=strictness)
    print(f"\n{strictness.upper()} Mode:")
    print(f"  Compliance: {result['scores']['overall_compliance']:.1f}%")
    print(f"  Status: {result['pass_fail_status']}")
    if result.get('violations'):
        print(f"  Violations: {len(result['violations'])}")

## Example 3: Iterative Refinement Based on Recommendations

When validation fails, the tool provides recommendations for improvement.

In [ ]:
# Initial response that may need refinement
response = "I think the answer is probably around 42, but I'm not entirely sure."

result = validate_text(response, strictness="strict")

print(f"Initial Compliance: {result['scores']['overall_compliance']:.1f}%")
print(f"Status: {result['pass_fail_status']}")

if result.get('violations'):
    print(f"\nViolations:")
    for violation in result['violations']:
        print(f"  - {violation}")

if result.get('recommendations'):
    print(f"\nRecommendations:")
    for rec in result['recommendations']:
        print(f"  - {rec}")

if result.get('strengths'):
    print(f"\nStrengths:")
    for strength in result['strengths']:
        print(f"  - {strength}")

## Example 4: Batch Validation

Validate multiple responses and compare their compliance scores.

In [ ]:
responses = [
    "The Earth orbits the Sun due to gravitational forces, completing one orbit approximately every 365.25 days.",
    "I believe the Earth probably goes around the Sun or something like that.",
    "Based on astronomical observations and Newtonian mechanics, Earth's orbital period is 365.256 days."
]

results = []
for i, response in enumerate(responses, 1):
    result = validate_text(response)
    results.append(result)
    print(f"\nResponse {i}:")
    print(f"  Compliance: {result['scores']['overall_compliance']:.1f}%")
    print(f"  Status: {result['pass_fail_status']}")
    print(f"  Truth Foundation Score: {result['scores']['law3_truth_foundation']:.1f}%")

## Example 5: Using Summary Format

The tool also supports a human-readable summary format.

In [ ]:
response = """Machine learning models require careful validation to ensure they generalize 
well to unseen data. Cross-validation and holdout sets are standard practices."""

summary = validate_text(response, format="summary")
print(summary)

## Example 6: Command-Line Usage

The tool can also be used directly from the command line.

In [ ]:
# Using command line with text argument
!python ../code/tools/run_five_laws_validator.py --text "The speed of light is approximately 299,792,458 meters per second." --format summary

In [ ]:
# Using stdin pipe
!echo "Water freezes at 0°C and boils at 100°C at standard atmospheric pressure." | python ../code/tools/run_five_laws_validator.py --format summary

## Example 7: Integration with AI Response Pipeline

Here's how to integrate the validator into an AI response generation pipeline.

In [ ]:
def generate_governed_response(prompt, max_iterations=3):
    """
    Generate an AI response and validate it against Five Laws.
    Refine until it passes or max iterations reached.
    """
    # Simulated AI response generation (replace with actual AI model)
    def generate_response(prompt):
        return f"Response to: {prompt}. This is a factual answer based on established scientific principles."
    
    for iteration in range(max_iterations):
        response = generate_response(prompt)
        result = validate_text(response, strictness="moderate")
        
        print(f"\nIteration {iteration + 1}:")
        print(f"  Compliance: {result['scores']['overall_compliance']:.1f}%")
        print(f"  Status: {result['pass_fail_status']}")
        
        if result['pass_fail_status'] == "PASS":
            print(f"  ✅ Response passed governance validation!")
            return response, result
        else:
            print(f"  ⚠️ Refinement needed")
            if result.get('recommendations'):
                print(f"  Recommendations: {result['recommendations'][0]}")
    
    print(f"\n⚠️ Max iterations reached. Returning best attempt.")
    return response, result

# Test the governed response pipeline
prompt = "Explain quantum entanglement"
final_response, validation = generate_governed_response(prompt)

## Conclusion

The Five Laws Validator is a powerful tool for ensuring AI-generated content meets cognitive governance standards. Key takeaways:

- Use **moderate** strictness for general-purpose validation
- Use **strict** mode for scientific, medical, or legal content
- Use **lenient** mode for creative or exploratory content
- Always review recommendations when validation fails
- Integrate validation into your AI pipeline for consistent governance

## Next Steps

- Explore other SIM-ONE tools: REP (Reasoning), ESL (Emotional State), VVP (Validation)
- Learn about protocol composition for complex workflows
- Read the full documentation at [PAPER2AGENT_INTEGRATION.md](../PAPER2AGENT_INTEGRATION.md)